In [1]:
import math
from typing import List, Optional, Tuple, Union
from abc import ABC, abstractmethod

import datasets
import torch
import numpy as np
import torch.nn as nn
import logging
import copy
import gc

from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    PreTrainedModel,
    PretrainedConfig,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    LlamaConfig,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    HfArgumentParser
)

In [2]:
from masks import (
    MergerConfig,
    Merger
)

In [3]:
merge_config = MergerConfig(
    model_paths = [
        # "/workspace/models/Arcee-VyLinh/",
        # "/workspace/models/Qwen2.5-Coder-3B/",
        "/workspace/models/L3.2-JametMini-3B-MK.III/",
        "/workspace/models/Llama-3.2-3B-Instruct-abliterated/"
    ],
    # mode = "vector_input",
    mode = "scalar"
)
merge_config

MergerConfig {
  "mode": "scalar",
  "model_paths": [
    "/workspace/models/L3.2-JametMini-3B-MK.III/",
    "/workspace/models/Llama-3.2-3B-Instruct-abliterated/"
  ],
  "transformers_version": "4.46.3"
}

In [4]:
merger = Merger(merge_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
merger = merger.to(device="cuda:0", dtype=torch.bfloat16)
merger.__post_init__(merge_config)

Initializing masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:57<00:00,  4.46it/s]
2024-12-18 09:12:29,005 - INFO - Initial GPU memory allocated: 11.97 GB
2024-12-18 09:12:29,367 - INFO - Final GPU memory allocated: 11.97 GB
2024-12-18 09:12:29,368 - INFO - Freed GPU memory: 0.00 GB
